# Gramática

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
PARSER PURO (sem lexer) + AST + desenho de árvore (opcional).

Extensões implementadas:
- Expressões com precedência: OR (||), AND (&&), ==, !=, <, <=, >, >=, +, -, *, /
- Pós-fixos: chamada de função f(a,b) e indexação arr[i]
- Statements: let, assign (ID = E), if/else, while, return, bloco { ... }
- Dois modos de parsing: parse_line (compatível com EOL) e parse_program (até EOF)

Gramática (visão geral):

# ---------------------------------------------------
# Estrutura de linhas e statements
# ---------------------------------------------------
Linha        ::= (Stmt (';' Stmt)*)? EOL
# Uma linha pode conter zero ou mais statements separados por ';'
# e deve terminar com fim de linha (EOL).

Stmt         ::= LetDecl | Assign | If | While | Return | E
# Um statement pode ser: declaração, atribuição, if, while, return ou apenas uma expressão.

LetDecl      ::= 'let' ID '=' E
# Declaração de variável com inicialização obrigatória.
# Ex.: let x = 10

Assign       ::= ID '=' E
# Atribuição de valor a uma variável já existente.
# Ex.: x = y + 2

If           ::= 'if' '(' E ')' Block ('else' Block)?
# Estrutura condicional com bloco obrigatório no "if"
# e bloco opcional no "else".
# Ex.: if (x > 0) { ... } else { ... }

While        ::= 'while' '(' E ')' Block
# Laço de repetição com condição entre parênteses e corpo em bloco.
# Ex.: while (x < 10) { ... }

Return       ::= 'return' E?
# Retorno de função. Pode ter expressão (return x + 1)
# ou ser vazio (return).

Block        ::= '{' StmtList? '}' | Stmt
# Bloco de código: ou um conjunto de statements entre { }
# ou um único statement sem chaves.

StmtList     ::= Stmt (';' Stmt)*
# Lista de statements separados por ';'.

# ---------------------------------------------------
# Expressões (precedência e associatividade)
# ---------------------------------------------------

E            ::= Or
# Expressão geral começa no nível mais alto: operações lógicas "ou".

Or           ::= And ( '||' And )*
# Operadores "OU lógico" (||), associativos à esquerda.
# Ex.: a || b || c

And          ::= Equality ( '&&' Equality )*
# Operadores "E lógico" (&&), associativos à esquerda.
# Ex.: a && b && c

Equality     ::= Relational ( ( '==' | '!=' ) Relational )*
# Comparações de igualdade e diferença.
# Ex.: a == b, a != b

Relational   ::= Add ( ( '<' | '<=' | '>' | '>=' ) Add )*
# Comparações relacionais (<, <=, >, >=).
# Ex.: x < 10, y >= z

Add          ::= Mul ( ( '+' | '-' ) Mul )*
# Operadores aritméticos de soma e subtração, associativos à esquerda.
# Ex.: a + b - c

Mul          ::= Postfix ( ( '*' | '/' ) Postfix )*
# Operadores aritméticos de multiplicação e divisão.
# Ex.: a * b / c

Postfix      ::= Primary ( '(' ArgList? ')' | '[' E ']' )*
# Pós-fixos possíveis sobre um "Primary":
# - chamada de função: f(x, y)
# - indexação em vetor: arr[2]
# (pode repetir várias vezes, ex.: f(x)(y)[z] )

Primary      ::= NUM | TRUE | FALSE | ID | '(' E ')'
# Valores atômicos: números, booleanos, identificadores ou expressão entre parênteses.

ArgList      ::= E (',' E)*
# Lista de argumentos de chamada de função, separados por vírgula.
"""

'\nPARSER PURO (sem lexer) + AST + desenho de árvore (opcional).\n\nExtensões implementadas:\n- Expressões com precedência: OR (||), AND (&&), ==, !=, <, <=, >, >=, +, -, *, /\n- Pós-fixos: chamada de função f(a,b) e indexação arr[i]\n- Statements: let, assign (ID = E), if/else, while, return, bloco { ... }\n- Dois modos de parsing: parse_line (compatível com EOL) e parse_program (até EOF)\n\nGramática (visão geral):\n\n# ---------------------------------------------------\n# Estrutura de linhas e statements\n# ---------------------------------------------------\nLinha        ::= (Stmt (\';\' Stmt)*)? EOL\n# Uma linha pode conter zero ou mais statements separados por \';\'\n# e deve terminar com fim de linha (EOL).\n\nStmt         ::= LetDecl | Assign | If | While | Return | E\n# Um statement pode ser: declaração, atribuição, if, while, return ou apenas uma expressão.\n\nLetDecl      ::= \'let\' ID \'=\' E\n# Declaração de variável com inicialização obrigatória.\n# Ex.: let x = 10\n\nA

In [2]:

from __future__ import annotations
from dataclasses import dataclass
from typing import List, Optional, Union, Any, Tuple, Dict
import os

 ## CLASSES DA ÁRVORE SINTÁTICA ABSTRATA (AST)

In [3]:

from dataclasses import is_dataclass

# --- Nós de Expressão ---
@dataclass
class Num:
    value: int
    token: Token  # Rastreia o token original para reportar erros

@dataclass
class Bool:
    value: bool
    token: Token

@dataclass
class Var:
    name: str
    token: Token

@dataclass
class BinOp:
    op: str
    left: 'Expr'
    right: 'Expr'
    token: Token

@dataclass
class Call:
    callee: 'Expr'
    args: List['Expr']
    token: Token

@dataclass
class Index:
    target: 'Expr'
    index: 'Expr'
    token: Token

# --- Nós de Statement ---
@dataclass
class Let:
    lhs: Var
    init: 'Expr'

@dataclass
class Assign:
    target: Var
    value: 'Expr'

@dataclass
class If:
    test: 'Expr'
    then: 'Block'
    otherwise: Optional['Block']

@dataclass
class While:
    test: 'Expr'
    body: 'Block'

@dataclass
class Return:
    value: Optional['Expr']

@dataclass
class Block:
    body: List['Stmt']

# --- Nó Raiz da Árvore ---
@dataclass
class Program:
    body: List['Stmt']

# --- Tipos auxiliares para anotação ---
Expr = Union[Num, Bool, Var, BinOp, Call, Index]
Stmt = Union[Let, Assign, If, While, Return, Block, Expr]

## CLASSE PARSER

In [ ]:
class Parser:
    def __init__(self, tokens: List[Token]):
        self.tokens = tokens
        self.pos = 0
        self.errors: List[str] = []

    def current_token(self) -> Token:
        """Retorna o token atual sem consumi-lo."""
        if self.pos < len(self.tokens):
            return self.tokens[self.pos]
        # Para evitar erros de índice no final da lista, retornamos o último token
        # (geralmente EOF) se a posição já tiver ultrapassado o limite.
        return self.tokens[-1]

    def advance(self) -> Token:
        """Consome o token atual e avança para o próximo."""
        token = self.current_token()
        if self.pos < len(self.tokens):
            self.pos += 1
        return token

    def expect(self, expected_type: str, message: Optional[str] = None) -> Optional[Token]:
        """
        Verifica se o token atual é do tipo esperado.
        - Se for, consome o token e o retorna.
        - Se não for, registra um erro, entra em modo pânico e retorna None.
        """
        token = self.current_token()
        if token.type == expected_type:
            return self.advance()

        # ERRO! Prepara a mensagem de erro padronizada.
        if not message:
            # Constrói uma mensagem de erro útil como "Esperado '='".
            # O código base pede para usar 'expect' para padronizar as mensagens.
            expected_lex = f"'{expected_type.lower()}'"
            if expected_type == "EQUAL": expected_lex = "'='"
            if expected_type == "LPAREN": expected_lex = "'('"
            if expected_type == "RPAREN": expected_lex = "')'"
            # E assim por diante para outros tokens...
            message = f"Esperado {expected_lex}"

        self.errors.append(f"{message} (encontrado '{token.lex}') @ {token.line}:{token.col}")

        # Ativa o modo pânico para tentar continuar a análise
        self.synchronize()
        return None

    def synchronize(self):
        """
        Modo Pânico: avança até encontrar um ponto seguro para recomeçar.
        Isso nos permite reportar múltiplos erros em uma única vez.
        """
        self.advance() # Consome o token que causou o erro
        while self.current_token().type != "EOF":
            # Para se estivermos sobre um token de início de statement
            if self.tokens[self.pos - 1].type in ["SEMI", "EOL"]:
                return
            # Para ao encontrar um token que provavelmente inicia um novo statement.
            if self.current_token().type in ["LET", "IF", "WHILE", "RETURN", "LBRACE", "RBRACE"]:
                return
            # Avança para o próximo token
            self.advance()


 # MÉTODOS DE PARSING DE OPERADORES (DENTRO DA CLASSE PARSER)

    def parse_mul(self) -> Optional[Expr]:
        """Analisa multiplicação e divisão (precedência mais alta)."""
        node = self.parse_postfix() # Começa com a precedência imediatamente superior
        if not node: return None

        while self.current_token().type in ("STAR", "SLASH"):
            op_token = self.advance()
            right = self.parse_postfix()
            if not right: return None # Propaga o erro
            node = BinOp(op=op_token.lex, left=node, right=right, token=op_token)
        return node

    def parse_add(self) -> Optional[Expr]:
        """Analisa soma e subtração."""
        node = self.parse_mul() # Chama o nível de precedência acima
        if not node: return None

        while self.current_token().type in ("PLUS", "MINUS"):
            op_token = self.advance()
            right = self.parse_mul()
            if not right: return None
            node = BinOp(op=op_token.lex, left=node, right=right, token=op_token)
        return node

    def parse_relational(self) -> Optional[Expr]:
        """Analisa operadores relacionais <, <=, >, >="""
        node = self.parse_add()
        if not node: return None
        
        while self.current_token().type in ("LT", "LE", "GT", "GE"):
            op_token = self.advance()
            right = self.parse_add()
            if not right: return None
            node = BinOp(op=op_token.lex, left=node, right=right, token=op_token)
        return node

    def parse_equality(self) -> Optional[Expr]:
        """Analisa operadores de igualdade ==, !="""
        node = self.parse_relational()
        if not node: return None

        while self.current_token().type in ("EQ", "NE"):
            op_token = self.advance()
            right = self.parse_relational()
            if not right: return None
            node = BinOp(op=op_token.lex, left=node, right=right, token=op_token)
        return node

    def parse_and(self) -> Optional[Expr]:
        """Analisa o operador lógico AND (&&)"""
        node = self.parse_equality()
        if not node: return None

        while self.current_token().type == "AND":
            op_token = self.advance()
            right = self.parse_equality()
            if not right: return None
            node = BinOp(op=op_token.lex, left=node, right=right, token=op_token)
        return node

    def parse_or(self) -> Optional[Expr]:
        """Analisa o operador lógico OR (||) (precedência mais baixa)."""
        node = self.parse_and()
        if not node: return None

        while self.current_token().type == "OR":
            op_token = self.advance()
            right = self.parse_and()
            if not right: return None
            node = BinOp(op=op_token.lex, left=node, right=right, token=op_token)
        return node

    def parse_e(self) -> Optional[Expr]:
        """Ponto de entrada para qualquer expressão. Começa da menor precedência."""
        # Note que já tínhamos uma chamada para este método dentro de parse_primary.
        # Agora estamos definindo-o oficialmente.
        return self.parse_or()

# Lógica para a construção da árvore (vocês podem modificar, caso necessário - é apenas uma dica)

In [5]:
# -----------------------------------------------------------
# Visualizador de AST genérico (duck typing) para salvar
# a árvore em PNG. Compatível com a AST do seu parser.
# -----------------------------------------------------------

try:
    import matplotlib.pyplot as plt
    HAVE_MPL = True
except Exception:
    HAVE_MPL = False

NodeLike = Any  # aceitamos qualquer objeto com atributos esperados

# ----------------------------
# Rótulo amigável para cada nó
# ----------------------------
def node_label(n: NodeLike) -> str:
    tname = type(n).__name__
    # Casos comuns da sua AST:
    if tname == "Program": return "Program"
    if tname == "Let":     return "Let"
    if tname == "Assign":  return "Assign"
    if tname == "If":      return "If"
    if tname == "While":   return "While"
    if tname == "Return":  return "Return"
    if tname == "Block":   return "Block"
    if tname == "Call":    return "Call"
    if tname == "Index":   return "Index"
    if tname == "BinOp":   return f"BinOp('{getattr(n, 'op', '?')}')"
    if tname == "Var":     return f"Id({getattr(n, 'name', '?')})"
    if tname == "Num":     return f"Num({getattr(n, 'value', '?')})"
    if tname == "Bool":
        v = getattr(n, "value", None)
        return f"Bool({str(v).lower()})" if isinstance(v, bool) else "Bool(?)"
    # Fallback genérico:
    return tname

# ----------------------------------------
# Lista de filhos (sem depender de imports)
# ----------------------------------------
def children(n: NodeLike) -> List[NodeLike]:
    tname = type(n).__name__
    # Mapeamento pelos atributos usados na sua AST
    if tname == "Program": return list(getattr(n, "body", []))
    if tname == "Let":     return [getattr(n, "lhs", None), getattr(n, "init", None)]
    if tname == "Assign":  return [getattr(n, "target", None), getattr(n, "value", None)]
    if tname == "If":
        lst = [getattr(n, "test", None), getattr(n, "then", None)]
        other = getattr(n, "otherwise", None)
        if other is not None: lst.append(other)
        return lst
    if tname == "While":   return [getattr(n, "test", None), getattr(n, "body", None)]
    if tname == "Return":
        v = getattr(n, "value", None)
        return [v] if v is not None else []
    if tname == "Block":   return list(getattr(n, "body", []))
    if tname == "Call":    return [getattr(n, "callee", None)] + list(getattr(n, "args", []))
    if tname == "Index":   return [getattr(n, "target", None), getattr(n, "index", None)]
    if tname == "BinOp":   return [getattr(n, "left", None), getattr(n, "right", None)]

    # Fallback: se for dataclass, tenta varrer campos;
    # caso contrário, tenta um atributo 'children' se existir.
    if is_dataclass(n):
        out: List[Any] = []
        for k, v in n.__dict__.items():
            if k in ("line", "col", "op", "name", "value"):  # metadados/escalares
                continue
            if isinstance(v, list):
                out.extend(v)
            elif v is not None:
                out.append(v)
        return out
    if hasattr(n, "children"):
        return list(getattr(n, "children"))
    return []

# -------------------------------------------------
# Layout recursivo (retorna posicoes e largura)
# -------------------------------------------------
def _compute_layout(n: NodeLike, x0=0.0, y0=0.0, y_spacing=1.6) -> Tuple[Dict[int,Tuple[float,float]], float]:
    ch = [c for c in children(n) if c is not None]
    if not ch:
        return ({id(n): (x0, y0)}, 1.0)

    pos: Dict[int,Tuple[float,float]] = {}
    widths: List[float] = []
    subs: List[NodeLike] = []

    for c in ch:
        subpos, w = _compute_layout(c, 0, 0, y_spacing)
        pos.update(subpos)
        widths.append(w)
        subs.append(c)

    total_w = sum(widths) + (len(widths)-1)*0.8
    cur_x = x0 - total_w/2.0

    def shift(node: NodeLike, dx: float, dy: float):
        x, y = pos[id(node)]
        pos[id(node)] = (x + dx, y + dy)
        for cc in children(node):
            if cc is not None:
                shift(cc, dx, dy)

    for c, w in zip(subs, widths):
        cx = cur_x + w/2.0
        shift(c, cx, y0 - y_spacing)
        cur_x += w + 0.8

    pos[id(n)] = (x0, y0)
    return pos, total_w

# -----------------------------------------
# Função principal: salva a árvore em PNG
# -----------------------------------------
def draw_tree(root: NodeLike, filename: str, figsize=(10, 7), dpi: int = 160):
    pos, _ = _compute_layout(root, 0.0, 0.0)

    fig, ax = plt.subplots(figsize=figsize)
    ax.set_axis_off()

    def draw_edges(node: NodeLike):
        x, y = pos[id(node)]
        for c in children(node):
            if c is None:
                continue
            xc, yc = pos[id(c)]
            ax.plot([x, xc], [y-0.05, yc+0.05])
            draw_edges(c)

    def draw_nodes(node: NodeLike):
        x, y = pos[id(node)]
        bbox = dict(boxstyle="round,pad=0.3", fc="white", ec="black", lw=1)
        ax.text(x, y, node_label(node), ha="center", va="center", bbox=bbox, fontsize=10)
        for c in children(node):
            if c is not None:
                draw_nodes(c)

    draw_edges(root)
    draw_nodes(root)

    xs = [xy[0] for xy in pos.values()]
    ys = [xy[1] for xy in pos.values()]
    pad = 1.2
    ax.set_xlim(min(xs)-pad, max(xs)+pad)
    ax.set_ylim(min(ys)-pad, max(ys)+pad)
    plt.tight_layout()
    plt.savefig(filename, dpi=dpi, bbox_inches="tight")
    plt.close(fig)

# Exemplo: usando seu parser
# program, errors = parse(tokens) # aqui é o retorno do seu parser
# if not errors:
#     draw_tree(program, "arvore_programa.png")


# Testes que você deverá executar

In [6]:
@dataclass
class Token:
    type: str
    lex: str
    line: int
    col: int

# ---------------- Casos de teste ----------------
# VÁLIDOS (5)

tokens1 = [
    # let y = 1 + 3;
    Token("LET","let",1,1),
    Token("ID","y",1,5),
    Token("EQUAL","=",1,7),
    Token("NUM","1",1,9),
    Token("PLUS","+",1,11),
    Token("NUM","3",1,13),
    Token("SEMI",";",1,14),
    Token("EOL","",1,15)
]
desc1 = "Atribuição simples com soma"

tokens2 = [
    # 2 * (3 + 4);
    Token("NUM","2",1,1),
    Token("STAR","*",1,3),
    Token("LPAREN","(",1,5),
    Token("NUM","3",1,6),
    Token("PLUS","+",1,8),
    Token("NUM","4",1,10),
    Token("RPAREN",")",1,11),
    Token("SEMI",";",1,12),
    Token("EOL","",1,13)
]
desc2 = "Expressão aritmética com parênteses"

tokens3 = [
    # let x = 5; x / 2
    Token("LET","let",1,1),
    Token("ID","x",1,5),
    Token("EQUAL","=",1,7),
    Token("NUM","5",1,9),
    Token("SEMI",";",1,10),
    Token("ID","x",1,12),
    Token("SLASH","/",1,14),
    Token("NUM","2",1,16),
    Token("EOL","",1,17)
]
desc3 = "Duas instruções na mesma linha (let e depois expressão)"

tokens4 = [
    # if (x < 10 && y != 0) { x = x + 1; } else x = 0;
    Token("IF","if",1,1),
    Token("LPAREN","(",1,3),
    Token("ID","x",1,4),
    Token("LT","<",1,6),
    Token("NUM","10",1,9),
    Token("AND","&&",1,11),
    Token("ID","y",1,14),
    Token("NE","!=",1,16),
    Token("NUM","0",1,19),
    Token("RPAREN",")",1,20),
    Token("LBRACE","{",1,22),
    Token("ID","x",1,24),
    Token("EQUAL","=",1,26),
    Token("ID","x",1,28),
    Token("PLUS","+",1,30),
    Token("NUM","1",1,32),
    Token("SEMI",";",1,33),
    Token("RBRACE","}",1,35),
    Token("ELSE","else",1,37),
    Token("ID","x",1,42),
    Token("EQUAL","=",1,44),
    Token("NUM","0",1,46),
    Token("SEMI",";",1,47),
    Token("EOL","",1,48)
]
desc4 = "if/else com && e !="

tokens5 = [
    # let z = f(a, b)[i] * g();
    Token("LET","let",1,1),
    Token("ID","z",1,5),
    Token("EQUAL","=",1,7),
    Token("ID","f",1,9),
    Token("LPAREN","(",1,10),
    Token("ID","a",1,11),
    Token("COMMA",",",1,12),
    Token("ID","b",1,14),
    Token("RPAREN",")",1,15),
    Token("LBRACK","[",1,16),
    Token("ID","i",1,17),
    Token("RBRACK","]",1,18),
    Token("STAR","*",1,20),
    Token("ID","g",1,22),
    Token("LPAREN","(",1,23),
    Token("RPAREN",")",1,24),
    Token("SEMI",";",1,25),
    Token("EOL","",1,26)
]
desc5 = "Chamada e indexação (sem unário)"

# INVÁLIDOS (6)

tokens6 = [
    # ERRO: faltou '=' após ID -> let z 7;
    Token("LET","let",1,1),
    Token("ID","z",1,5),
    # Token("EQUAL","=",1,7),  # ausente
    Token("NUM","7",1,9),
    Token("SEMI",";",1,10),
    Token("EOL","",1,11)
]
desc6 = "ERRO: faltou '=' na atribuição"

tokens7 = [
    # ERRO: parêntese não fechado -> 1 + (2 * 3;
    Token("NUM","1",1,1),
    Token("PLUS","+",1,3),
    Token("LPAREN","(",1,5),
    Token("NUM","2",1,6),
    Token("STAR","*",1,8),
    Token("NUM","3",1,10),
    # faltou RPAREN
    Token("SEMI",";",1,11),
    Token("EOL","",1,12)
]
desc7 = "ERRO: parêntese aberto sem fechar com ')'"

tokens8 = [
    # ERRO: 'else' sem 'if'
    Token("ELSE","else",1,1),
    Token("ID","x",1,6),
    Token("EQUAL","=",1,8),
    Token("NUM","1",1,10),
    Token("SEMI",";",1,11),
    Token("EOL","",1,12)
]
desc8 = "ERRO: 'else' sem 'if' correspondente"

tokens9 = [
    # ERRO: while com parêntese da condição não fechado
    # while (x < 5 { x = x + 1; }
    Token("WHILE","while",1,1),
    Token("LPAREN","(",1,7),
    Token("ID","x",1,8),
    Token("LT","<",1,10),
    Token("NUM","5",1,12),
    # faltou RPAREN
    Token("LBRACE","{",1,14),
    Token("ID","x",1,16),
    Token("EQUAL","=",1,18),
    Token("ID","x",1,20),
    Token("PLUS","+",1,22),
    Token("NUM","1",1,24),
    Token("SEMI",";",1,25),
    Token("RBRACE","}",1,27),
    Token("EOL","",1,28)
]
desc9 = "ERRO: while sem fechar ')' da condição"

tokens10 = [
    # ERRO: indexador sem ']' -> let a = arr[1 + 2;
    Token("LET","let",1,1),
    Token("ID","a",1,5),
    Token("EQUAL","=",1,7),
    Token("ID","arr",1,9),
    Token("LBRACK","[",1,12),
    Token("NUM","1",1,13),
    Token("PLUS","+",1,15),
    Token("NUM","2",1,17),
    # faltou RBRACK
    Token("SEMI",";",1,18),
    Token("EOL","",1,19)
]
desc10 = "ERRO: indexação sem ']'"

tokens11 = [
    Token("LET","let",1,1),
    Token("ID","a",1,5),
    # Token("EQUAL","=",1,7),  # faltando de propósito
    Token("NUM","1",1,7),
    Token("SEMI",";",1,8),

    Token("ID","x",1,10),
    Token("EQUAL","=",1,12),
    # faltou expressão aqui
    Token("SEMI",";",1,13),

    Token("ID","y",1,15),
    Token("EQUAL","=",1,17),
    Token("NUM","3",1,19),
    Token("SEMI",";",1,20),
    Token("EOL","",1,21),
]
desc11 = "Dois erros, mas parser continua e aceita último stmt válido"

In [7]:
# Runner
CASES = [
    ("case1_let_y",            desc1,  tokens1),
    ("case2_expr_paren",       desc2,  tokens2),
    ("case3_two_stmts",        desc3,  tokens3),
    ("case4_if_else_logic",    desc4,  tokens4),
    ("case5_call_index",       desc5,  tokens5),
    ("case6_missing_equal",    desc6,  tokens6),
    ("case7_missing_rparen",   desc7,  tokens7),
    ("case8_lonely_else",      desc8,  tokens8),
    ("case9_while_rparen",     desc9,  tokens9),
    ("case10_missing_rbrack",  desc10, tokens10),
    ("case11_two_errors_same_line", desc11, tokens11),
]

if __name__ == "__main__":
    print("=== SUÍTE DE TESTES DO PARSER ===")
    for name, desc, toks in CASES:
        print(f"\n>>> {name}")

=== SUÍTE DE TESTES DO PARSER ===

>>> case1_let_y

>>> case2_expr_paren

>>> case3_two_stmts

>>> case4_if_else_logic

>>> case5_call_index

>>> case6_missing_equal

>>> case7_missing_rparen

>>> case8_lonely_else

>>> case9_while_rparen

>>> case10_missing_rbrack

>>> case11_two_errors_same_line


# Esperado



```
=== SUÍTE DE TESTES DO PARSER ===

>>> case1_let_y

======================================================================
[CASO] case1_let_y
[RESULTADO] OK — AST construída
[ÁRVORES] salvas em 'trees/case1_let_y_stmtNN.png'

>>> case2_expr_paren

======================================================================
[CASO] case2_expr_paren
[RESULTADO] OK — AST construída
[ÁRVORES] salvas em 'trees/case2_expr_paren_stmtNN.png'

>>> case3_two_stmts

======================================================================
[CASO] case3_two_stmts
[RESULTADO] OK — AST construída
[ÁRVORES] salvas em 'trees/case3_two_stmts_stmtNN.png'

>>> case4_if_else_logic

======================================================================
[CASO] case4_if_else_logic
[RESULTADO] OK — AST construída
[ÁRVORES] salvas em 'trees/case4_if_else_logic_stmtNN.png'

>>> case5_call_index

======================================================================
[CASO] case5_call_index
[RESULTADO] OK — AST construída
[ÁRVORES] salvas em 'trees/case5_call_index_stmtNN.png'

>>> case6_missing_equal

======================================================================
[CASO] case6_missing_equal
[RESULTADO] ERROS SINTÁTICOS
  - Esperado '=' (encontrado '7') @ 1:9
[AST (parcial)]

>>> case7_missing_rparen

======================================================================
[CASO] case7_missing_rparen
[RESULTADO] ERROS SINTÁTICOS
  - Esperado ')' (encontrado ';') @ 1:11
[AST (parcial)]

>>> case8_lonely_else

======================================================================
[CASO] case8_lonely_else
[RESULTADO] ERROS SINTÁTICOS
  - Esperado número, 'true', 'false', identificador, '(' (encontrado 'else') @ 1:1
[AST (parcial)]

>>> case9_while_rparen

======================================================================
[CASO] case9_while_rparen
[RESULTADO] ERROS SINTÁTICOS
  - Esperado ')' (encontrado '{') @ 1:14
  - Esperado número, 'true', 'false', identificador, '(' (encontrado '}') @ 1:27
[AST (parcial)]

>>> case10_missing_rbrack

======================================================================
[CASO] case10_missing_rbrack
[RESULTADO] ERROS SINTÁTICOS
  - Esperado ']' (encontrado ';') @ 1:18
[AST (parcial)]

>>> case11_two_errors_same_line

======================================================================
[CASO] case11_two_errors_same_line
[RESULTADO] ERROS SINTÁTICOS
  - Esperado '=' (encontrado '1') @ 1:7
  - Esperado número, 'true', 'false', identificador, '(' (encontrado ';') @ 1:13
[AST (parcial)]
```

